In [1]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from sklearn.metrics import classification_report
import numpy as np

In [2]:
# Define the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))  # Assuming 10 genres

d:\SDS\AUDIO CLASSIFICATION\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [3]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:
print("CNN model initialized successfully.")

CNN model initialized successfully.


In [5]:
# Define directories
train_dir = r'D:\SDS\AUDIO CLASSIFICATION\Spectrograms\train'
test_dir = r'D:\SDS\AUDIO CLASSIFICATION\Spectrograms\test'

In [6]:
# Check if the directories exist and are not empty
if not os.path.exists(train_dir):
    raise ValueError(f"Directory {train_dir} does not exist.")
if not os.listdir(train_dir):
    raise ValueError(f"Directory {train_dir} is empty.")

if not os.path.exists(test_dir):
    raise ValueError(f"Directory {test_dir} does not exist.")
if not os.listdir(test_dir):
    raise ValueError(f"Directory {test_dir} is empty.")

In [7]:
# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True)

In [8]:
# Normalization for testing
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Found 799 images belonging to 10 classes.
Found 200 images belonging to 10 classes.


d:\SDS\AUDIO CLASSIFICATION\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step - accuracy: 0.1119 - loss: 2.3656 - val_accuracy: 0.2396 - val_loss: 2.0911
Epoch 2/20
 1/24 ━━━━━━━━━━━━━━━━━━━━ 7s 313ms/step - accuracy: 0.2188 - loss: 2.0255

d:\SDS\AUDIO CLASSIFICATION\.venv\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 96ms/step - accuracy: 0.2188 - loss: 2.0255 - val_accuracy: 0.2083 - val_loss: 2.1433
Epoch 3/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 17s 683ms/step - accuracy: 0.2568 - loss: 2.0550 - val_accuracy: 0.2396 - val_loss: 1.9532
Epoch 4/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - accuracy: 0.2500 - loss: 2.0290 - val_accuracy: 0.2448 - val_loss: 1.9241
Epoch 5/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 17s 686ms/step - accuracy: 0.3327 - loss: 1.8820 - val_accuracy: 0.3906 - val_loss: 1.7750
Epoch 6/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.3438 - loss: 1.8850 - val_accuracy: 0.3490 - val_loss: 1.7706
Epoch 7/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 16s 678ms/step - accuracy: 0.3461 - loss: 1.7656 - val_accuracy: 0.3646 - val_loss: 1.7183
Epoch 8/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - accuracy: 0.4062 - loss: 1.7338 - val_accuracy: 0.3854 - val_loss: 1.7144
Epoch 9/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 16s 674ms/step - accuracy: 0.4201 - loss: 1.6471 - val_accuracy: 0.3073 - val_

In [9]:
# Predict the classes for the test set
Y_pred = model.predict(validation_generator, validation_generator.samples // validation_generator.batch_size + 1)
y_pred = np.argmax(Y_pred, axis=1)

7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 289ms/step


In [10]:
# Get the true classes
y_true = validation_generator.classes

In [11]:
# Print the classification report
print('Classification Report')
target_names = list(validation_generator.class_indices.keys())
print(classification_report(y_true[:len(y_pred)], y_pred, target_names=target_names))

Classification Report
              precision    recall  f1-score   support

       blues       0.12      0.05      0.07        20
   classical       0.13      0.15      0.14        20
     country       0.00      0.00      0.00        20
       disco       0.08      0.25      0.12        20
      hiphop       0.00      0.00      0.00        20
        jazz       0.09      0.05      0.06        20
       metal       0.21      0.15      0.18        20
         pop       0.00      0.00      0.00        20
      reggae       0.05      0.10      0.06        20
        rock       0.00      0.00      0.00        20

    accuracy                           0.07       200
   macro avg       0.07      0.07      0.06       200
weighted avg       0.07      0.07      0.06       200



d:\SDS\AUDIO CLASSIFICATION\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\SDS\AUDIO CLASSIFICATION\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\SDS\AUDIO CLASSIFICATION\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
